In [ ]:
#######################################################################
# MED 263:Bioinformatics Applications to Human Disease (Winter 2025)  #
# Analysis of epigenetic data for precision cancer diagnosis          #
# Lukas Chavez, Feb 2025                                              #
# UC San Diego                                                        #
#######################################################################

In [ ]:
library(minfi) # Package for loading and preprocessing methylation data from many sources including Illumina. See https://bioconductor.org/packages/devel/bioc/vignettes/minfi/inst/doc/minfi.html
library(minfiData) # Example dataset for minfi vignette.
library(conumee) # Package for estimating genomic copy number from methylation. See https://bioconductor.org/packages/devel/bioc/vignettes/conumee/inst/doc/conumee.html
library(Rtsne) # t-stochastic neighbor embedding.
library(sva) # one of many batch correction algorithms
library(tictoc) # System time functions tic() and toc()

In [ ]:
#Identify Illumina Infinium raw data (idat files)
# Set manually:
baseDir <- system.file("extdata", package = "minfiData")

In [ ]:
#Warm-up exercises:
########################
#1.1) Show the location/path and content of baseDir. What are the R commands?
#1.2) How many arrays and how many samples are in the example data?
###########################

#None of the following will work when minfi and minfidata cannot be installed

#Import sample sheet
targets <- minfi::read.metharray.sheet(baseDir)

#Import entire metharray experiment using a sample sheet and returns an RGChannelSet (RGSet) object
RGSet <- minfi::read.metharray.exp(targets = targets)

#Sample information
phenoData <- Biobase::pData(RGSet)

#The manifest contains the information and annotation of the probes on the array
manifest <- minfi::getManifest(RGSet)

In [ ]:
# You can view documentation using ?
class(RGSet) # get the type of a variable
?RGChannelSet # view the documentation of that type

In [ ]:
#1.3) Fill in the numbers:
###########################
#Number of type I probes:  
#Number of type II probes:  
#Number of control probes:  
###########################

#Creation of a MethylSet without normalization
MSet <- preprocessRaw(RGSet) 

#Show matrix of methylation values
#methylated channel
head(getMeth(MSet)[,1:6])
dim(getMeth(MSet))

#unmethylated channel
head(getUnmeth(MSet)[,1:6])
dim(getUnmeth(MSet))

#Converting methylation data from methylation and unmethylation channels, to ratios (Beta-values).
#Returns a ratioSet
RSet <- ratioConvert(MSet, what = "both", keepCN = TRUE)

#Extracts beta values
beta <- getBeta(RSet)

head(beta)

In [ ]:
########################
#1.4) How many probes are on the array? Is the example data 450k or 850k data?
###########################

#Map methylation data to the genome
#Returns a GenomicRatioSet
GRset <- mapToGenome(RSet)

#Create new beta matrix based on ordered probes mapped to the genome
beta <- getBeta(GRset)

#Genomic locations of probes
gr <- granges(GRset)

In [ ]:
########################
#1.5) What is the "cg" ID of the first probe on chromosome 1 (chr1, position 15,865)
###########################

#Access sample information from GRset
sampleNames <- sampleNames(GRset)
probeNames <- featureNames(GRset)
pheno <- pData(GRset)

#getQC/ matrixStats function deprecated
#Estimate sample-specific quality control (QC) for methylation data
#qc <- getQC(MSet)
#plotQC(qc)

#Distribution of beta values
densityPlot(MSet, sampGroups = phenoData$Sample_Group)
densityBeanPlot(MSet, sampGroups = phenoData$Sample_Group)

In [ ]:
########################
#1.6) One of the two groups (Group A and Group B) seem to have a slight increase in probes that have an intermediate methylation (0.2-0.6 beta). 
# Are those the normal or the cancer samples?
###########################

#getSeq/ matrixStats function deprecated
#Biological sex prediction
#med(X) = median total intensity of the X chromosome-mapped probes
#med(Y) = median total intensity of the Y chromosome-mapped probes
#predictedSex <- getSex(GRset, cutoff = -2)


In [ ]:
########################
#SKIP - getSex/ matrixStats deprecated
#1.x) Is sample 5723646053_R05C02 likely male or female? 
#1.x) True or false: The ratio of median measurements on the two sex chromosomes (xMed, yMed) is closer to 1 for male samples.
###########################

In [ ]:
#################################################################
# Exercise II
# Stratification of patient medulloblastoma tumors by
# clustering of Illumina Infinium DNA methylation array data
#################################################################
#Data published by Northcott et al., The whole-genome landscape of medulloblastoma subtypes, Nature 2017
#Raw microarray available (access controlled, don't distribute) at European Genome-Phenome Archive (EGA, http://www.ebi.ac.uk/ega/), under accession number EGAS00001001953.

In [ ]:
# Load preprocessed beta values of 1,256 medulloblastoma patients stored 
# in the file MBlandscape_completed_unique.RData (provided with course material)
tic("Load MB dataset")
load(url("https://datasets.genepattern.org/data/chapman/MBlandscape_subset_400.RData")) #load the object 'allbeta'
# load("data/MBlandscape_subset_400.RData") # for a local dataset stored in ./data
toc()

In [ ]:
# For each CpG, calculate the standard deviation across the cohort and
# order the CpGs according to their standard deviation (from top to bottom)
# This takes awhile
tic("Order CpG sites by variance")
allbeta.sd <- apply(allbeta, 1, sd, na.rm=TRUE)
allbeta.ordered <- allbeta[order(allbeta.sd, decreasing=TRUE),]
head(allbeta.ordered)
toc()

In [ ]:
#Calculate Pearson correlation between all tumors based
#on the 5k most variable CpGs
tic("Sample correlation heatmap")
n.var = 5000
b <- allbeta.ordered[1:n.var, ]

#Caluclate Pearson correlations between samples/patients
b.xcor <- cor(b, method="pearson")

# perform hierarchical sample clustering
b.xdend <- as.dendrogram(hclust(as.dist(1-b.xcor), method="average"))

# Visualisation of pre-computed hierarchical sample/patient clustering
cols <- colorRampPalette(c("blue", "white", "red"))(100)
heatmap(b.xcor, Rowv=b.xdend, col=cols, symm=TRUE, zlim=c(-1, 1), scale="none", useRaster=TRUE,
        cexCol=max(min(125*ncol(b)^-1.25, 1), 0.07), labRow=NA, main=paste(dim(b), collapse="x"))
#--> symmetric matrix of similarities between patients
toc()
# 8 min on my M2 2023 macbook pro

In [ ]:
# TSNE -  t-distributed stochastic neigborhood embedding
# non-linear dimension reduction
tic("t-sne")
set.seed("202401")
Y <- Rtsne(as.dist(1-b.xcor), verbose=FALSE, check_duplicates=FALSE, is_distance=TRUE,
             perplexity=min(floor((ncol(b)-1)/3), 30), theta=0, pca=FALSE, max_iter=10000)$Y
Y.range <- apply(Y, 2, range)
Y.diff <- apply(Y.range, 2, diff)
Y.center <- apply(Y.range, 2, mean)
plot(Y, xlim=Y.center[1] + c(-0.5, 0.5)*max(Y.diff), ylim=Y.center[2] + c(-0.5, 0.5)*max(Y.diff),
     xlab="TSNE 1", ylab="TSNE 2", pch=20, cex=1, col="black", main=paste(dim(b), collapse="x"), las=2)
toc()

In [ ]:
########################
#1.7) Cluster the medulloblastoma samples/patients into four subtypes using k-means and replot the t-sne by assigning distinct colors according to the k-means clusters.
###########################
 